<h1><center>English to French Language Translation using Encoders and Decoders</center></h1>

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import string
from string import digits
import re

from sklearn.model_selection import train_test_split

datapath = '/content/drive/MyDrive/eng_fra.txt'

In [6]:
df = pd.read_table(datapath, names=['eng', 'fr'])

In [7]:
data = df[:30000]

In [8]:
data.sample(5)

,eng,fr
25505,He used to love her.,Il l'avait aimée.
2993,Bring him in.,Fais-le entrer.
28982,Why are you staying?,Pourquoi restes-tu ?
2650,Tom's funny.,Tom est marrant.
22520,Let's hit the road.,On se casse !


In [9]:
data = data.applymap(lambda x: x.lower())

In [10]:
data.eng=data.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
data.fr=data.fr.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [11]:
exclude = set(string.punctuation)
data.eng=data.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
data.fr=data.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [12]:
data.head()

,eng,fr
0,go,va
1,run,cours
2,run,courez
3,wow,ça alors
4,fire,au feu


In [13]:
remove_digits = str.maketrans('', '', digits)
data=data.applymap(lambda x: x.translate(remove_digits))

In [14]:
data.fr = data.fr.apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
data.sample()

,eng,fr
7700,im on holiday,START_ je suis en vacances _END


In [16]:
all_eng_words = set()
for eng in data.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_fra_words = set()
for fra in data.fr:
    for word in fra.split():
        if word not in all_fra_words:
            all_fra_words.add(word)

In [17]:
len(all_eng_words),len(all_fra_words)

(4467, 9327)

In [18]:
# Max Length of source sequence
length_list_eng=[]
for sent in data.eng:
    length_list_eng.append(len(sent.split(' ')))
max_length_src = np.max(length_list_eng)

# Max Length of target sequence
length_list_fra=[]
for sent in data.fr:
    length_list_fra.append(len(sent.split(' ')))
max_length_tar = np.max(length_list_fra)

In [19]:
max_length_src,max_length_tar

(7, 14)

In [20]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_fra_words))

num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_fra_words)

num_encoder_tokens,num_decoder_tokens

(4467, 9327)

In [21]:
num_encoder_tokens+=1
num_decoder_tokens+=1

In [22]:
input_token_index=dict([(word,i+1) for i, word in enumerate(input_words)])
target_token_index=dict([(word,i+1) for i, word in enumerate(target_words)])

In [23]:
reverse_input_token_index=dict((i,word) for word, i in input_token_index.items())
reverse_target_token_index=dict((i,word) for word, i in target_token_index.items())

In [24]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head(10)

,eng,fr
26284,i only think of you,START_ je ne pense quà toi _END
21546,i like mathematics,START_ jaime les maths _END
108,go away,START_ dégage _END
21979,im always careful,START_ je suis toujours prudente _END
16503,he did not listen,START_ il nécouta pas _END
13405,i love this game,START_ jadore cette manche _END
23546,tom likes swimming,START_ tom aime nager _END
22534,lets wait a while,START_ attendons un moment _END
22894,stop dating losers,START_ arrête de sortir avec des tocards _END
27160,its very different,START_ cest très différent _END


In [25]:
X, y = data['eng'], data['fr']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((24000,), (6000,))

In [26]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [27]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [42]:
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [43]:
latent_dim=300

In [44]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [45]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [46]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [47]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    1340400     ['input_6[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 300)    2798400     ['input_7[0][0]']                
                                                                                            

In [48]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 20

In [49]:
pd.DataFrame(X_train).max()

eng    youve worked hard
dtype: object

I have took the help of generators as I can't fit my entire training data into RAM, as it is throwing out of memory exception.
I have used here `model.fit_generator`, we can also use `model.fit` as `Model.fit_generator` is deprecated and Model.fit() itself supports generators

In [50]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

<ipython-input-50-b22c3ec5e69e>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/20
187/187 [==============================] - 279s 1s/step - loss: 5.9630 - val_loss: 5.2596
Epoch 2/20
187/187 [==============================] - 277s 1s/step - loss: 5.1385 - val_loss: 5.0662
Epoch 3/20
187/187 [==============================] - 262s 1s/step - loss: 4.8953 - val_loss: 4.8076
Epoch 4/20
187/187 [==============================] - 275s 1s/step - loss: 4.6441 - val_loss: 4.5823
Epoch 5/20
187/187 [==============================] - 274s 1s/step - loss: 4.4050 - val_loss: 4.3538
Epoch 6/20
187/187 [==============================] - 276s 1s/step - loss: 4.1647 - val_loss: 4.1473
Epoch 7/20
187/187 [==============================] - 256s 1s/step - loss: 3.9624 - val_loss: 3.9943
Epoch 8/20
187/187 [==============================] - 258s 1s/step - loss: 3.7932 - val_loss: 3.8632
Epoch 9/20
187/187 [==============================] - 276s 1s/step - loss: 3.6431 - val_loss: 3.7485
Epoch 10/20
187/187 [==============================] - 280s 1s/step - loss: 3.5080 - val_lo

In [51]:
from tensorflow.keras.models import model_from_json
model.save('/content/drive/MyDrive/lang_trans_model.h5')

In [52]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/lang_trans_model.h5')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    1340400     ['input_6[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 300)    2798400     ['input_7[0][0]']                
                                                                                            

In [53]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 300)         1340400   
                                                                 
 lstm_2 (LSTM)               [(None, 300),             721200    
                              (None, 300),                       
                              (None, 300)]                       
                                                                 
Total params: 2,061,600
Trainable params: 2,061,600
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [55]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_token_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [56]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [66]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 34ms/step
Input English sentence: you are a good boy
Actual French Translation:  vous êtes un bon garçon 
Predicted French Translation:  vous êtes un bon garçon 


In [76]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 67ms/step
Input English sentence: where is my son
Actual French Translation:  où est mon fils  
Predicted French Translation:  où est mon voiture 


In [77]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 31ms/step
Input English sentence: you look busy
Actual French Translation:  vous avez lair occupés 
Predicted French Translation:  vous avez lair heureux 


In [64]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 28ms/step
Input English sentence: she likes sleeping
Actual French Translation:  elle apprécie de dormir 
Predicted French Translation:  elle adore les yeux 
